# Recommend For New Users

## 1. Simple Filter & Sort

    -- Choose genre and recommend movies according to the rating.

In [2]:
# import pandas as pd
# import numpy as np

In [5]:
# # read movies data
# movies = pd.read_csv('ml-20m/movies.csv')
# movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# movies.shape

(27278, 3)

In [7]:
# len(set(movies.movieId))

27278

In [37]:
# from itertools import chain
# x = [s.split('|') for s in movies.genres]
# g_list = set(chain(*x))
# g_list

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

### 1.1 Recommend according to ratings

In [8]:
# ratings = pd.read_csv('ml-20m/ratings.csv')
# ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [9]:
# len(set(ratings.movieId))

26744

In [61]:
# ratio of rating > 4
len(ratings[ratings['rating']>4])/len(ratings)

0.22167128502260194

22% people rates above 4. So we can think it is a relatively good cutoff of recommending or not.

In [10]:
mean = pd.DataFrame(ratings[['movieId','rating']].groupby(['movieId'])['rating'].mean())
mean['movieId'] = mean.index
mean.index = range(len(mean))

In [11]:
mean.head()

,rating,movieId
0,3.921240,1
1,3.211977,2
2,3.151040,3
3,2.861393,4
4,3.064592,5


In [12]:
merged_table = pd.merge(movies, mean, on = 'movieId')

In [13]:
merged_table.head()

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.921240
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.211977
2,3,Grumpier Old Men (1995),Comedy|Romance,3.151040
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.861393
4,5,Father of the Bride Part II (1995),Comedy,3.064592


In [14]:
merged_table.shape

(26744, 4)

In [25]:
result = merged_table.sort_values('rating', ascending = False)
def genre_choose_rough(genre):
    final = result[result.genres.str.contains('|'.join(genre))].title[:10].tolist()
    return final

In [26]:
inp = ['Adventure']
genre_choose_rough(inp)

['Giorgino (1994)',
 'Life On A String (Bian chang Bian Zou) (1991)',
 'Victor and the Secret of Crocodile Mansion (2012)',
 'Into the Middle of Nowhere (2010)',
 'Stargate SG-1 Children of the Gods - Final Cut (2009)',
 'The Beautiful Story (1992)',
 'Curse of the Ring (Ring of the Nibelungs) (2004)',
 'The Magnificent Gladiator (1964)',
 'Symphony of the Soil (2012)',
 'Itinerary of a Spoiled Child (1988)']

### 1.2 Take the number of ratings for a single film into account

In [18]:
count = pd.DataFrame(ratings[['movieId','userId']].groupby(['movieId'])['userId'].count())
count['movieId'] = count.index
count.index = range(len(count))

In [19]:
count.columns = ['count','movieId']
count = count.sort_values('count', ascending = False)
count.head()

,count,movieId
293,67310,296
352,66172,356
315,63366,318
587,63299,593
476,59715,480


In [20]:
count.describe()

,count,movieId
count,26744.000000,26744.000000
mean,747.841123,58789.612436
std,3085.818268,44159.344871
min,1.000000,1.000000
25%,3.000000,6795.750000
50%,18.000000,65880.000000
75%,205.000000,99110.500000
max,67310.000000,131262.000000


In [21]:
len(count[count['count'] < 5])

8399

In [22]:
merged = pd.merge(merged_table, count, on = 'movieId')
merged.head()

,movieId,title,genres,rating,count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.921240,49695
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.211977,22243
2,3,Grumpier Old Men (1995),Comedy|Romance,3.151040,12735
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.861393,2756
4,5,Father of the Bride Part II (1995),Comedy,3.064592,12161


In [49]:
# check if all genres contain 10 movies with rating>4 
result = merged.sort_values('count', ascending = False)

for x in g_list:
    y = result[result.genres.str.contains(str(x)) & (result.rating > 4)]
    print(x, len(y))

/Users/ying/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


(no genres listed) 20
Adventure 70
War 62
Mystery 61
Documentary 186
Action 73
Crime 92
Film-Noir 28
Musical 28
Animation 35
Romance 126
Sci-Fi 38
Western 10
IMAX 6
Comedy 230
Horror 19
Thriller 95
Fantasy 40
Children 23
Drama 494


In [68]:
# rating > 4 and sorted in number of ratings
result = merged.sort_values('count', ascending = False)

def choose(genre):
    
    inpt = '|'.join(genre)
    final = result[result.genres.str.contains(inpt) & (result.rating > 4)]
    
    if len(final) < 10:
        return final.title.tolist()
    
    return final.title[:10].tolist()


In [69]:
choose(['IMAX'])

['Dark Knight, The (2008)',
 'Inception (2010)',
 'Toy Story 3 (2010)',
 'Dark Knight Rises, The (2012)',
 'How to Train Your Dragon (2010)',
 'Interstellar (2014)']

In [65]:
choose(['Adventure','Action'])

['Star Wars: Episode IV - A New Hope (1977)',
 'Braveheart (1995)',
 'Matrix, The (1999)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Fight Club (1999)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Saving Private Ryan (1998)',
 'Lord of the Rings: The Two Towers, The (2002)']

In [62]:
我感觉这部分可以不要，10个评价还是太少了
# # the number of ratings > 10 and sorted in rating
# inpt = 'Adventure'
# result = merged.sort_values('rating', ascending = False)
# result[result.genres.str.contains(inpt) & (result['count'] > 10)].head()#.title[:10].tolist()

## 2. Content-Based Recommendation

    -- Recommend for a typical movie

In [21]:
# genome_scores = pd.read_csv('Project91/ml-20m/genome-scores.csv')
# genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [22]:
# genome_scores.shape

(11709768, 3)

In [23]:
# len(set(genome_scores.movieId))

10381

In [24]:
# max(genome_scores.tagId)

1128

In [25]:
# transform the genome_scores dataframe into relevance matrix
df = genome_scores.groupby(['movieId'])['relevance'].apply(list).apply(pd.Series)
score_matrix = np.matrix(df)

In [26]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,0.03200,...,0.03950,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300
2,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,0.02050,...,0.04175,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875
3,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,0.02675,...,0.04150,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850
4,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,0.03025,...,0.05750,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300
5,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,0.02875,...,0.04250,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825


In [27]:
df.shape

(10381, 1128)

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

# compute cosine similarity for every movie 
cosine_sim = cosine_similarity(score_matrix, score_matrix)

In [29]:
# get the movieId value and reset the df index
df['movieId']=df.index
df.index = range(len(df))

In [30]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1119,1120,1121,1122,1123,1124,1125,1126,1127,movieId
0,0.02500,0.02500,0.05775,0.09675,0.14675,0.21700,0.06700,0.26275,0.26200,0.03200,...,0.01800,0.04575,0.03275,0.12500,0.04150,0.01925,0.03625,0.07775,0.02300,1
1,0.03975,0.04375,0.03775,0.04800,0.11025,0.07250,0.04775,0.10975,0.09925,0.02050,...,0.01925,0.01725,0.02425,0.12550,0.02250,0.01550,0.01475,0.09025,0.01875,2
2,0.04350,0.05475,0.02800,0.07700,0.05400,0.06850,0.05600,0.18500,0.04925,0.02675,...,0.02675,0.02775,0.03425,0.15550,0.03675,0.01700,0.01950,0.09700,0.01850,3
3,0.03725,0.03950,0.03675,0.03100,0.06825,0.04050,0.02325,0.08700,0.05125,0.03025,...,0.03375,0.02275,0.03975,0.18525,0.05925,0.01500,0.01525,0.06450,0.01300,4
4,0.04200,0.05275,0.05925,0.03675,0.07525,0.12525,0.02850,0.08500,0.02950,0.02875,...,0.02825,0.02150,0.02600,0.14275,0.02075,0.01650,0.01675,0.10750,0.01825,5


In [31]:
def recommendation(title):
    
    # link title to movieId and find the row number of the matrix
    ids = movies[movies['title'] == title]['movieId'].values[0]
    i = df[df.movieId==ids].index.values[0]
    
    # find the specific score for the movie and sort the scores
    sim_scores = list(enumerate(cosine_sim[i]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # store the row numbers and related movieId
    indices = [i[0] for i in sim_scores[1:]]
    movie_ids = df.iloc[indices,:].movieId.values
    
    result = []
    for i in movie_ids:
        result.append(movies[movies.movieId == i].title.values[0])
    
    return result

In [32]:
# recommend 10 movies
recommendation('Toy Story (1995)')[:10]

['Monsters, Inc. (2001)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Finding Nemo (2003)',
 'Toy Story 3 (2010)',
 'Ratatouille (2007)',
 'Ice Age (2002)',
 'Shrek (2001)',
 'Up (2009)',
 'Antz (1998)']